In [6]:
#이미지 업로드
#파일 이름 변경할 것

from google.colab import files

uploaded = files.upload() 
for name, data in uploaded.items():
  
  with open(name, 'wb') as f:
    f.write(data)
    print ('saved file', name)

Saving style_1.jpg to style_1.jpg
saved file style_1.jpg


In [2]:
from keras.preprocessing.image import load_img, img_to_array, save_img

target_image_path = 'content_1.jpg'
style_reference_image_path = 'style_1.jpg'

# 생성된 사진의 차원
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)



Using TensorFlow backend.


In [3]:
import cv2
import numpy
myimg = cv2.imread('content_1.jpg')
avg_color_per_row = numpy.average(myimg, axis=0)
avg_color = numpy.average(avg_color_per_row, axis=0)
print(avg_color)

[ 84.90021364 113.09404273 139.5465842 ]


In [4]:
#유틸리티 함수 정의

import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
  img = load_img(image_path, target_size = (img_height, img_width))
  img = img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = vgg19.preprocess_input(img)
  return img

def deprocess_image(x):
  x[:, :, 0] += avg_color[0]
  x[:, :, 1] += avg_color[1]
  x[:, :, 2] += avg_color[2]

  x = x[:, :, ::-1]
  x = np.clip(x, 0, 255).astype('unit8') #0보다 작은 값을 0으로 대체
  return x

In [7]:
# 훈련된 VGG19 네트워크를 로딩하고 3개 이미지에 적용하기
from keras import backend 

target_image = backend.constant(preprocess_image(target_image_path))
style_reference_image = backend.constant(preprocess_image(style_reference_image_path))
combination_image = backend.placeholder((1, img_height, img_width, 3)) #생성된 이미지를 담을 placeholder

input_tensor = backend.concatenate([target_image, style_reference_image, combination_image], axis = 0) #3개의 이미지를 하나로 합친다.

model = vgg19. VGG19(input_tensor = input_tensor,
                     weights = 'imagenet',
                     include_top = False)


80142336/80134624 [==============================] - 2s 0us/step


In [8]:
# 콘텐츠 손실
def content_loss(base, combination):
  return backend.sum(backend.square(combination -  base))

In [9]:
# 스타일 손실
def gram_matrix(x):
  features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
  gram = backend.dot(features, backend.transpose(features))
  return gram

def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = size = img_height * img_width
  return backend.sum(backend.square(S-C))/ (4. * (channels **2) * (size**2))


In [10]:
# 총 변위 손실
def total_variation_loss(x):
  a = backend.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
  b = backend.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height -1, 1:, :])

  return backend.sum(backend.pow(a + b, 1.25))

In [16]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers]) #층 이름과 활성화 텐서를 매핑한 딕셔너리

content_layer = 'block5_conv2'
style_layers = ['block1_conv2', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1'] # 스타일 손실에 사용할 층

total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = backend.variable(0.) #모든 손실 요소를 더하여 하나의 스칼라 변수로 손실을 정의함
layer_features = outputs_dict[content_layer]

target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
# loss += content_weight * content_loss(target_image_features, combination_features)
loss.assign_add(content_weight * content_loss(target_image_features, combination_features))
for layer_name in style_layers:
  layer_features = outputs_dict[layer_name]
  style_reference_features = layer_features[1, :, :, :]
  combination_features = layer_features[2, :, :, :]
  sl = style_loss(style_reference_features, combination_features)
  loss.assign_add((style_weight / len(style_layers)) * sl)

loss.assign_add(total_variation_weight * total_variation_loss(combination_image))

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=0.0>

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>

In [17]:
#경사 하강법 단계 설정하기

grads = backend.gradients(loss, combination_image)[0] #손실에 대한 생성된 이미지의 경사도를 구합니다.

fetch_loss_and_grads = backend.function([combination_image], [loss, grads]) #손실과 경사도 값을 추출하는 케라스 함수 객체

class Evaluator(object):

  def __init__(self):
    self.loss_value = None
    self.grads_values = None
  
  def loss(self, x):
    assert self.loss_value is None
    x = x.reshape((1, img_height, img_width, 3))
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grads_values = outs[1].flatten().astype('float64')
    self.loss_value = loss_value
    self.grad_values = grad_values
    return self.loss_value

  def grads(self, x):
    assert self.loss_value is not None
    grad_values = np.copy(self.grad_values)
    self.loss_value = None
    self.grad_vales = None

    return grad_values

evaluator = Evaluator()

AttributeError: ignored